# **通过子类化定义新层和模型**

### **引入**

In [1]:
import tensorflow as tf
from tensorflow import keras

### `Layer`类：状态（权重）和一些计算的组合体

`Layer`类是Keras的核心抽象类之一。层封装了状态（层的“权重”）和从输入到输出的转换（“调用”，即层的前向传递）。

下面是一个密度连接的层。它具有一个状态：变量`w`和`b`。

In [2]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

你可以通过在某些张量输入上调用层来使用它，就像使用Python函数一样。

In [3]:
x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.03713623  0.08639161 -0.02649306 -0.05403492]
 [ 0.03713623  0.08639161 -0.02649306 -0.05403492]], shape=(2, 4), dtype=float32)


请注意，权重`w`和`b`在设置为层的属性时会自动由层跟踪：

In [4]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

请注意，你还可以使用一种更快的方法为层添加权重：`add_weight()`方法：

In [5]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.07010797  0.00309221  0.00851233  0.04159135]
 [-0.07010797  0.00309221  0.00851233  0.04159135]], shape=(2, 4), dtype=float32)


### **层可以拥有不可训练的权重**

除了可训练的权重之外，你还可以向层添加不可训练的权重。在训练层时，不能再反向传播期间使用这类权重。

以下是添加和使用不可训练的权重的方法：

In [6]:
class ComputeSum(keras.layers.Layer):
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)), trainable=False)

    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total


x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


它是`layer.weights`的一部分，但被归类为不可训练的权重：

In [7]:
print("weights:", len(my_sum.weights))
print("non-trainable weights:", len(my_sum.non_trainable_weights))

# 不包含可训练权重:
print("trainable_weights:", my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []


### 最佳实践：得到输入的形状后，再创建权重

我们上面的`Linear`层采用了`input_dim`参数，用于计算`__init__()`中权重`w`和`b`的形状：

In [8]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

在许多情况下，你可能事先不知道输入的大小，并且你想在实例化层后的某个时间，得到输入大小后再创建权重。

在Keras API中，建议你在层的`build(self，inputs_shape)`方法中创建层的权重。像如下示例这样：

In [9]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

层的`__call__()`方法在首次调用时将自动执行`build`。现在你有了一个懒惰的层，因此更易于使用：

In [10]:
# 实例化时，我们不知道使用何种输入进行调用
linear_layer = Linear(32)

# 首次调用该层时，将动态创建该层的权重
y = linear_layer(x)

### **层可以递归组合**

如果将层实例作为另一个层的属性，则外层将会开始跟踪内层的权重。

我们建议在`__init__()`方法中创建这样的子层（由于子层通常具有构建方法，因此将在外层构建时构建它们）。

In [11]:
# 我们将上面定义了“build”方法的Linear类一起使用。


class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6


### **add_loss()方法**

编写层的`call()`方法时，可以创建损失张量，随后将在编写训练循环时使用。这可以通过调用`self.add_loss(value)`来实现：

In [12]:
# 产生活动正则化损失的层
class ActivityRegularizationLayer(keras.layers.Layer):
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate

    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

这些损失（包括由所有内部层产生的损失）可以通过`layer.losses`进行检索。此属性在顶层的`__call__()`开始时重置，因此`layer.losses`始终包含在上一次正向传递过程中创建的损失值。

In [13]:
class OuterLayer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)

    def call(self, inputs):
        return self.activity_reg(inputs)


layer = OuterLayer()
assert len(layer.losses) == 0  # 由于从未调用过该层，因此尚未产生任何损失

_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # 创建一个损失值

# `layer.losses`在__call__开始时被重置
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1  # 这是上面调用中造成的损失

此外，`loss`属性还包含为所有内层的权重创建的正则化损失：

In [14]:
class OuterLayerWithKernelRegularizer(keras.layers.Layer):
    def __init__(self):
        super(OuterLayerWithKernelRegularizer, self).__init__()
        self.dense = keras.layers.Dense(
            32, kernel_regularizer=tf.keras.regularizers.l2(1e-3)
        )

    def call(self, inputs):
        return self.dense(inputs)


layer = OuterLayerWithKernelRegularizer()
_ = layer(tf.zeros((1, 1)))

# 下面的值由`1e-3 * sum(layer.dense.kernel ** 2)`计算得来,
# 即通过上面的`kernel_regularizer`创建的.
print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0022100222>]


In [ ]:
在编写训练循环时应考虑这些损失，如下所示：

In [ ]:
# 实例化一个优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 遍历数据集的批量。
for x_batch_train, y_batch_train in train_dataset:
  with tf.GradientTape() as tape:
    logits = layer(x_batch_train)  # 小批量的Logits
    # 小批量的损失值
    loss_value = loss_fn(y_batch_train, logits)
    # 添加在前向传递中产生的额外损失：
    loss_value += sum(model.losses)

  grads = tape.gradient(loss_value, model.trainable_weights)
  optimizer.apply_gradients(zip(grads, model.trainable_weights))

有关编写训练循环的详细指南，请参阅[从头开始编写训练循环的指南](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch/)。

这些损失还可以通过`fit()`无缝工作（它们会自动求和并添加到主要损失中）：

In [16]:
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# 如果在`compile`中传递了损失，则将正则化损失添加到其中
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# 也可以不给`compile`传递损失，因为模型已经在前向传递过程中调用`add_loss`使损失最小化了！
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

1/1 [==============================] - 0s 1ms/step - loss: 0.0376


### **add_metric()方法**

与`add_loss()`类似，层还具有`add_metric()`方法，用于在训练过程中跟踪指标的变化平均值。

考虑以下的示例：“LogisticEndpoint”层。它以预测和目标作为输入，通过`add_loss()`跟踪计算的损失，并通过`add_metric()`跟踪计算的精度标量。

In [17]:
class LogisticEndpoint(keras.layers.Layer):
    def __init__(self, name=None):
        super(LogisticEndpoint, self).__init__(name=name)
        self.loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
        self.accuracy_fn = keras.metrics.BinaryAccuracy()

    def call(self, targets, logits, sample_weights=None):
        # 计算训练时的损失，并使用`self.add_loss()`将其添加到层
        loss = self.loss_fn(targets, logits, sample_weights)
        self.add_loss(loss)

        # 记录精度指标，并使用`self.add_metric()`将其添加到层
        acc = self.accuracy_fn(targets, logits, sample_weights)
        self.add_metric(acc, name="accuracy")

        # 返回预测的张量(即`.predict()`).
        return tf.nn.softmax(logits)


可通过`layer.metrics`访问以这种方式跟踪的指标：

In [18]:
layer = LogisticEndpoint()

targets = tf.ones((2, 2))
logits = tf.ones((2, 2))
y = layer(targets, logits)

print("layer.metrics:", layer.metrics)
print("current accuracy value:", float(layer.metrics[0].result()))

layer.metrics: [<tensorflow.python.keras.metrics.BinaryAccuracy object at 0x7f4ee1e140f0>]
current accuracy value: 1.0


就像`add_loss()`一样，这些指标也可以通过`fit()`进行跟踪：

In [19]:
inputs = keras.Input(shape=(3,), name="inputs")
targets = keras.Input(shape=(10,), name="targets")
logits = keras.layers.Dense(10)(inputs)
predictions = LogisticEndpoint(name="predictions")(logits, targets)

model = keras.Model(inputs=[inputs, targets], outputs=predictions)
model.compile(optimizer="adam")

data = {
    "inputs": np.random.random((3, 3)),
    "targets": np.random.random((3, 10)),
}
model.fit(data)

1/1 [==============================] - 0s 1ms/step - loss: 0.9742 - binary_accuracy: 0.0000e+00


### **你可以选择性的启用层的序列化**

如果需要将自定义层作为函数式模型的一部分进行序列化，则可以选择实现`get_config()`方法：

In [20]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"units": self.units}


# 现在，你可以通过from_config重新创建该层：
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'units': 64}


In [21]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({"units": self.units})
        return config


layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'name': 'linear_8', 'trainable': True, 'dtype': 'float32', 'units': 64}


如果需要更大的灵活性，从其配置反序列化层时，则可以重写`from_config()`类方法来实现。下面是`from_config()`的基本实现：

In [ ]:
def from_config(cls, config):
  return cls(**config)

要了解有关序列化和保存的更多信息，请参阅有关[模型保存和序列化的完整指南](https://www.tensorflow.org/guide/keras/save_and_serialize/)。

### **call()方法中特殊的训练参数**

某些层，尤其是`BatchNormalization`层和`Dropout`层，在训练和预测期间具有不同的行为。 对于此类层，标准做法是在`call()`方法中公开训练（boolean）参数。

通过在`call()`中公开此参数，可以启用内置的训练和评估循环（例如`fit()`）以在训练和预测中正确使用该层。

In [22]:
class CustomDropout(keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

### **call()方法中特殊的掩码参数**

`call()`支持的另一个特殊参数是`mask`参数。

你可以在所有Keras RNN层中找到它。掩码是布尔张量（输入中每个时间步有一个布尔值），用于在处理时间序列数据时跳过某些输入时间步。

当先前的层生成`mask`时，Keras会自动将正确的`mask`参数传递给支持该层的`__call__()`。`mask`生成层是配置有`mask_zero = True`的“`Embedding`”层和“`Masking`”层。

要了解有关masking以及如何编写masking-enabled的层的更多信息，请查看指南“[理解padding和masking](https://www.tensorflow.org/guide/keras/masking_and_padding/)”。

### **Model类**

通常情况下，你将使用`Layer`类来定义内部计算块，并使用`Model`类来定义外部模型--你将训练的对象。

例如，在**ResNet50**模型中，你将有几个子类化`Layer`的**ResNet**，以及一个`Model`则包含了整个**ResNet50**网络。

`Model`类具有与`Layer`同样的API，但有以下区别：
+ 它公开了内置的训练，评估和预测循环（`model.fit()`，`model.evaluate()`，`model.predict()`）。
+ 它通过`model.layers`属性公开其内层的列表。
+ 它公开了保存和序列化API（`save()`，`save_weights()`...）

实际上，`Layer`类对应于我们在文献中所称的“层”（如“卷积层”或“递归层”）或“块”（如“ ResNet块”或“ Inception块”） ）。

同时，`Model`类对应于文献中所谓的“模型”（如“深度学习模型”）或“网络”（如“深度神经网络”）。

因此，如果你想知道“我应该使用`Layer`类还是`Model`类？”，请问自己：我需要在其上调用`fit()`吗？我需要在上面调用`save()`吗？如果需要，请选择`Model`。如果不需要（因为你的类只是更大系统中的一个块，或者是因为你自己编写训练和保存代码），请使用`Layer`。

例如，我们可以以上面的mini-resnet示例为例，并使用它来构建一个模型，该模型可以通过`fit()`进行训练，并可以通过`save_weights()`保存：

In [ ]:
class ResNet(tf.keras.Model):

    def __init__(self):
        super(ResNet, self).__init__()
        self.block_1 = ResNetBlock()
        self.block_2 = ResNetBlock()
        self.global_pool = layers.GlobalAveragePooling2D()
        self.classifier = Dense(num_classes)

    def call(self, inputs):
        x = self.block_1(inputs)
        x = self.block_2(x)
        x = self.global_pool(x)
        return self.classifier(x)


resnet = ResNet()
dataset = ...
resnet.fit(dataset, epochs=10)
resnet.save(filepath)

### 整合所有知识点：端到端的示例

+ 到目前为止，你已经学到了以下内容：
+ `Layer`封装状态（在`__init__()`或`build()`中创建）和一些计算（在`call()`中定义）
+ 可以递归嵌套层以创建更大的新计算块
+ 层可以通过`add_loss()`和`add_metric()`创建和跟踪损失（通常是正则化损失）以及指标
+ 你要训练的外部容器是一个`Model`。模型就像一个`Layer`，但是增加了训练和序列化功能

让我们将所有这些内容放到一个端到端的示例中：我们将实现一个变体自动编码器（VAE）。我们将用MNIST数字对其进行训练。

我们的VAE将是`Model`的子类，通过`Layer`的子类嵌套构建的。它将具有正则化损失（KL散度）。

In [23]:
from tensorflow.keras import layers


class Sampling(layers.Layer):
    """使用(z_mean，z_log_var)采样z，该向量编码一个数字。”"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """将MNIST数字映射到三元组(z_mean，z_log_var，z)"""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """将编码的数字矢量z转换回可读的数字"""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """将编码器和解码器整合到端到端模型以进行训练"""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # 添加KL散度作为正则化损失.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

让我们在MNIST上编写一个简单的训练循环：

In [24]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 2

# 遍历epochs.
for epoch in range(epochs):
    print("Start of epoch %d" % (epoch,))

    # 遍历dataset的批量
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # 计算重建损失
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # 添加KLD正则化
        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

11493376/11490434 [==============================] - 0s 0us/step
Start of epoch 0
step 0: mean loss = 0.3072
step 100: mean loss = 0.1247
step 200: mean loss = 0.0988
step 300: mean loss = 0.0889
step 400: mean loss = 0.0840
step 500: mean loss = 0.0807
step 600: mean loss = 0.0786
step 700: mean loss = 0.0770
step 800: mean loss = 0.0759
step 900: mean loss = 0.0749
Start of epoch 1
step 0: mean loss = 0.0746
step 100: mean loss = 0.0739
step 200: mean loss = 0.0734
step 300: mean loss = 0.0730
step 400: mean loss = 0.0726
step 500: mean loss = 0.0723
step 600: mean loss = 0.0719
step 700: mean loss = 0.0717
step 800: mean loss = 0.0714
step 900: mean loss = 0.0712


请注意，由于VAE是`Model`的子类，所以它具有内置的训练循环。因此，你也可以像这样训练它：

In [25]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=2, batch_size=64)

Epoch 1/2
938/938 [==============================] - 2s 2ms/step - loss: 0.0747
Epoch 2/2
938/938 [==============================] - 2s 2ms/step - loss: 0.0676


### **超越面向对象的开发：函数式API**

前面的示例对你来说，是不是使用了太多的面向对象开发？你也可以使用函数式API构建模型。重要的是，不论选择哪种风格不会妨碍你利用以另一种风格编写的组件：你始终可以混合搭配使用。

例如，下面的函数式API示例中，重用了我们在上面示例中定义的相同`Sampling`层：

In [26]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# 定义编码模型
original_inputs = tf.keras.Input(shape=(original_dim,), name="encoder_input")
x = layers.Dense(intermediate_dim, activation="relu")(original_inputs)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name="encoder")

# 定义解码模型
latent_inputs = tf.keras.Input(shape=(latent_dim,), name="z_sampling")
x = layers.Dense(intermediate_dim, activation="relu")(latent_inputs)
outputs = layers.Dense(original_dim, activation="sigmoid")(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name="decoder")

# 定义VAE模型
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name="vae")

# 添加KL散度为正则化损失
kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# 训练
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Epoch 1/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0746
Epoch 2/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0677
Epoch 3/3
938/938 [==============================] - 2s 2ms/step - loss: 0.0676


有关更多信息，请确保阅读[函数式API指南](https://www.tensorflow.org/guide/keras/functional/)。